In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))
from spider2_utils import load_csv_database

-setup-

In [ ]:
import pandas as pd
_database = load_csv_database("EntertainmentAgency", rows_limit=-1)
musical_preferences = _database["Musical_Preferences"]
musical_styles = _database["Musical_Styles"]

### Question

Could you list each musical style with the number of times it appears as a 1st, 2nd, or 3rd preference in a single row per style?

Step 1: Create separate columns for each preference (1st, 2nd, 3rd)


In [ ]:

ranked = pd.DataFrame({
    'FirstStyle': musical_preferences.loc[musical_preferences['PreferenceSeq'] == 1, 'StyleID'],
    'SecondStyle': musical_preferences.loc[musical_preferences['PreferenceSeq'] == 2, 'StyleID'],
    'ThirdStyle': musical_preferences.loc[musical_preferences['PreferenceSeq'] == 3, 'StyleID']
})

Step 2: Stack and count preferences


In [ ]:
counts = pd.DataFrame()

for col in ['FirstStyle', 'SecondStyle', 'ThirdStyle']:
    temp = ranked[[col]].dropna().value_counts().reset_index()
    temp.columns = ['StyleID', col]
    if counts.empty:
        counts = temp
    else:
        counts = counts.merge(temp, on='StyleID', how='outer')

Step 3: Fill missing counts with 0


In [ ]:
counts = counts.fillna(0).astype({'FirstStyle': int, 'SecondStyle': int, 'ThirdStyle': int})

Step 5: Merge with style names


In [ ]:
result = counts.merge(musical_styles, on='StyleID')

Step 6: Filter out rows where all counts are 0


In [ ]:
result = result[
    (result['FirstStyle'] > 0) |
    (result['SecondStyle'] > 0) |
    (result['ThirdStyle'] > 0)
]

Step 7: Sort and select result columns


In [ ]:
result = result.sort_values(
    by=['FirstStyle', 'SecondStyle', 'ThirdStyle', 'StyleID'],
    ascending=[False, False, False, True]
)[['StyleName', 'FirstStyle', 'SecondStyle', 'ThirdStyle']]
